In [9]:
# Importar todas las librerías necesarias.
import csv
import pandas as pd
import datetime
import os
import numpy as np
import random
import math
import pprint
import matplotlib.pyplot as plt

from htm.bindings.sdr import SDR, Metrics
from htm.encoders.rdse import RDSE, RDSE_Parameters
from htm.encoders.date import DateEncoder
from htm.bindings.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
from htm.bindings.algorithms import Predictor

In [10]:
# Cargar el archivo
_INPUT_FILE_PATH = os.path.join("/home/capri/thesis_msc/docs/files", "gymdata.csv")

# Default parameters in HTM
default_parameters = {
# there are 2 (3) encoders: "value" (RDSE) & "time" (DateTime weekend, timeOfDay)
 'enc': {
      "value" :
         {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
      "time": 
         {'timeOfDay': (30, 1), 'weekend': 21}
 },
 'predictor': {'sdrc_alpha': 0.1},
 'sp': {'boostStrength': 3.0,
        'columnCount': 1638,
        'localAreaDensity': 0.04395604395604396,
        'potentialPct': 0.85,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'synPermInactiveDec': 0.006},
 'tm': {'activationThreshold': 17,
        'cellsPerColumn': 13,
        'initialPerm': 0.21,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'minThreshold': 10,
        'newSynapseCount': 32,
        'permanenceDec': 0.1,
        'permanenceInc': 0.1},
 'anomaly': {
   'likelihood': 
       {'probationaryPct': 0.1,
        'reestimationPeriod': 100}
 }
}

In [20]:
# Read the CSV file
data = pd.read_excel('./docs/files/DL_Traffic_202006.xlsx', index_col=0)
print(data.iloc[2])

DL_Traffic (Gb)    13824.08
Name: 2020-06-01 00:30:00, dtype: float64
